In [1]:
import requests
import pandas as pd
import mysql.connector

In [32]:
# Configurações da API
base_url = "http://wapalternativa01.dalla.srv.br:3119/root"
token = "R3385Gti8C4qh45NqYbFe6kTTQ0hsNP03fx21w0u2"  # Substitua pelo seu token
headers = {
    "X-Token": token,
    "Content-Type": "application/json"
}

mysql_user="lamobuser"
mysql_password="AVNS_w50l8PbyCR09wodhFV3"
mysql_host="db-lamob-do-user-17950265-0.m.db.ondigitalocean.com"
mysql_database="dw_lamob"
mysql_port=25060

# Função para obter vendas da fila
def get_vendas():
    response = requests.get(f"{base_url}/venda", headers=headers)
    response.raise_for_status()
    return response.json()["venda"]  # Retorna a lista de vendas

# Função para remover a venda da fila
def remover_venda(codigo_venda):
    response = requests.post(f"{base_url}/venda/{codigo_venda}/ok", headers=headers)
    return response.status_code

# Função principal para processar todas as vendas até que a fila esteja vazia
def processar_todas_as_vendas():
    todas_as_vendas = []  # Para armazenar todas as vendas processadas

    while True:
        vendas = get_vendas()
        
        # if not vendas:
        #     print(f"Nenhuma venda restante na fila. {len(todas_as_vendas)} vendas processadas.")
        #     break

        
        for venda in vendas:
            codigo_venda = venda.get("Codigo")
            if codigo_venda:
                todas_as_vendas.append(venda)  # Adiciona a venda para salvar no banco
                print(f"Venda {codigo_venda}")
        
        break

    return todas_as_vendas

# Função para converter valores de dicionários para strings
def converter_valores_para_str(row):
    
    return [str(item) if isinstance(item, (dict, list)) else item for item in row]

# Função para salvar as vendas processadas no banco de dados PostgreSQL
def salvar_vendas_bd(df_novas_vendas):
    # Conectar ao banco de dados PostgreSQL
    conn = mysql.connector.connect(
        host=mysql_host,
        port=mysql_port,
        database=mysql_database,
        user=mysql_user,
        password=mysql_password
    )
    
    cursor = conn.cursor()
    
    # Insere os dados no banco de dados
    for index, row in df_novas_vendas.iterrows():
        row = converter_valores_para_str(row)

        column_names = ', '.join([col.lower() for col in df_novas_vendas.columns])
        placeholders = ', '.join(['%s'] * len(df_novas_vendas.columns))

        insert_query = f"INSERT INTO fact_vendas ({column_names}) VALUES ({placeholders})"
        cursor.execute(insert_query, list(row))
    
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Vendas processadas e salvas no banco de dados PostgreSQL")

def vendas_df():
    # Executa o processamento de todas as vendas
    vendas_processadas = processar_todas_as_vendas()

    # Convertendo os dados processados em um DataFrame
    df_novas_vendas = pd.DataFrame(vendas_processadas) 

    return df_novas_vendas

def execucao_full():
    # Salvando as vendas no banco de dados PostgreSQL
    salvar_vendas_bd(vendas_df())

    


In [33]:
execucao_full()

Venda 46554
Venda 47528
Venda 48046
Venda 48152
Venda 48169
Venda 48379
Venda 48462
Venda 48511
Venda 48525
Venda 49135
Venda 49270
Venda 49317
Venda 49352
Venda 50206
Venda 50642
Venda 51357
Venda 51368
Venda 51372
Venda 51379
Venda 51382
Venda 51385
Venda 51396
Venda 51421
Venda 51433
Venda 51440
Venda 51451
Venda 51459
Venda 51462
Venda 51463
Venda 51466
Venda 51467
Venda 51471
Venda 51483
Venda 51495
Venda 51513
Venda 51531
Venda 51536
Venda 51544
Venda 51551
Venda 51565
Venda 51567
Venda 51573
Venda 51578
Venda 51579
Venda 51599
Venda 51603
Venda 51610
Venda 51613
Venda 51628
Venda 51631
[46554, 305349, 'F', '10595875807', '', 16, '411.60', '0.00', '0.00', '0.00', '0.00', '2024-10-02', '2024-10-03', True, 'ALAIM DIAS', 'alaim2000009447795366@mercadolivre.com', '43908486138', '00000000', 'Avenida Ipanema', '901', 'RIO BRANCO VEICULOS', 'Vila Nova Sorocaba', 'SOROCABA', 'SP', '18070671', '', '0.00', 'LAMOBILIA FILIAL', '', 999, 'Em Conflito/Disputa', '2024-10-21T10:14:13.897-03:00',